In [20]:
import numpy as np
from numpy.linalg import norm


In [21]:
N = 10000
p = 10
K = 2
M = 50
n = 5000
seed = 0

In [22]:
np.random.seed(seed=seed)

In [23]:
# parameters - beta
beta = np.random.randn(K+1, p) * (-1)
beta[0] = 0

In [24]:
beta_norm = norm(beta)
print(beta_norm)
beta = beta / beta_norm

5.118144887785791


In [25]:
# features - X
X = np.random.randn(N, p)  # features
X[:, 0] = 1                # set the first columns of X to be constants

In [26]:
# true labels - Y
Y = np.argmax(X.dot(np.transpose(beta)), axis=1)

In [27]:
# pilot sample - X1 and Y1
from sklearn.model_selection import train_test_split

X1, X2, Y1, Y2 = train_test_split(X, Y, test_size=(N-n)/N, random_state=seed)

In [28]:
# annotation task assignment - A1
alpha = 1

A1 = np.random.binomial(1, alpha, size=(n, M))

In [29]:
# annotation probability - AP1
sigma_list = np.arange(1, M+1, 1)

AP1 = X1.dot(np.transpose(beta))
AP1 = AP1.reshape(AP1.shape + (1,))
AP1 = AP1 / sigma_list
AP1 = np.exp(AP1)

AP1_sum = AP1.sum(axis=1, keepdims=True)
AP1 = AP1 / AP1_sum

In [12]:
# annotation - AY1
AY1 = np.zeros((n, M))
for i in range(n):
    for m in range(M):
        prob_im = AP1[i, :, m]
        Y_im = np.argmax(np.random.multinomial(1, prob_im, 1))
        AY1[i, m] = Y_im

In [13]:
AY1[A1 == 0] = -1

In [14]:
AY1

array([[0., 2., 2., ..., 1., 0., 2.],
       [2., 2., 1., ..., 0., 0., 0.],
       [0., 1., 2., ..., 0., 1., 0.],
       ...,
       [2., 1., 2., ..., 2., 2., 2.],
       [2., 0., 2., ..., 2., 0., 0.],
       [2., 1., 1., ..., 0., 1., 1.]])

# Initial Estimator

In [15]:
from sklearn.linear_model import LogisticRegression

initial_b = np.zeros((M, K+1, p))
initial_beta = np.zeros((K+1, p))
initial_sigma = np.zeros(M)

In [16]:
for m in range(M):
    y = AY1[:, m]
    clf = LogisticRegression(random_state=0, fit_intercept=False).fit(X1, y)
    initial_b[m] = clf.coef_
    initial_sigma[m] = 1 / norm(initial_b[m])
    initial_beta += initial_b[m] * initial_sigma[m] / M

In [17]:
norm(initial_beta - beta)

0.7413100764781911

# One-Step Estimator

In [18]:
X1.shape, AY1.shape

((5000, 10), (5000, 50))

In [19]:
K

2